# Container Apps - RBAC only
This template sets up a containerized application environment with necessary Azure resources, including a managed identity, Azure Container Registry access, container deployment, optional storage account, and specified role assignments.

## Prerequisites
- Azure CLI installed
- Bicep CLI installed
- Python
- Jupyter Notebook or JupyterLab installed
- Container app environment ([See scenario-container-app-environment.ipynb](../environment/scenario-container-app-environment.ipynb))


## Setup Environment
1. Make an .env file, configure the settings on your needs: 

In [1]:
%%writefile .env
SUBSCRIPTION_ID=<subscription_id>
RESOURCE_GROUP=<resource_group>
LOCATION=eastus

Writing .env


2. Install `python-dotenv` if you haven't already. You can install it using pip:

In [ ]:
!pip install python-dotenv

3. Load the settings in the environments:

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
location = os.getenv('LOCATION')

template_file = "main.bicep"
parameters_file = "parameters.json"

print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Location: {location}")

Subscription ID: <subscription_id>
Resource Group: <resource_group>
Location: eastus


4. Create an Azure Resource group:

In [3]:
!az login
!az account set --subscription  {subscription_id}
!az group create --name {resource_group} --location eastus

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "id": "835ebe2e-68d1-4817-925d-1aae42ec01db",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "37ac251a-9341-4ae8-b2a2-aa7febf5ce05"
      }
    ],
    "name": "RBR_DPT_GENERIC_P",
    "state": "Enabled",
    "tenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "user": {
      "name": "ROB5207@robeco.nl",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "id": "e84d14c9-e150-4500-93b2-5aa2829e295d",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "37ac251a-9341-4ae8-b2a2-aa7febf5ce05"
      }
    ],
    "name": "RBR_CSS_EQUITY_O",
    "state": "Enabled",
    "tenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "user": {
      "name": "ROB5207@robeco.nl",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "7

The syntax of the command is incorrect.
The system cannot find the file specified.


## Deploy template
1. Set the bicep parameters, adjust the settings when needed:

In [ ]:
%%writefile main.bicepparam

using 'main.bicep'

param containerAppEnvName = 'my-container-app-env'
param containerAppEnvRG = 'container-apps'

param acrName = 'lenndewoltentestacr'
param acrRG = 'container-apps'

param containerAppName = 'my-test-container-app'
param storageAccountName = 'lenntstchtrappfilestrg'

param image = {
  repository: 'containerapps-helloworld'
  tag: 'latest'
}

param resources = {
  cpu: '0.5'
  memory: '1Gi'
}


2. Optional: Seed container registry with the docker image

In [ ]:
acrName = 'lenndewoltentestacr'
repository = 'containerapps-helloworld'
tag = 'latest'

!az acr import --name {acrName} --source mcr.microsoft.com/azuredocs/containerapps-helloworld:latest --image {repository}:{tag}

3. Deploy the template:

In [ ]:
!az deployment group create --resource-group {resource_group} --template-file main.bicep  --parameters main.bicepparam --name container-apps-deployment 


4. Get the url:

In [ ]:
!az deployment group show --resource-group {resource_group} --name container-apps-deployment --query "properties.outputs.applicationUrl.value" --output tsv